# <div align="center"><h2>Фильтры калмана в компьютерном зрении</h2> <h3>пример поиска скрытой переменной (скорости) без доказательств <br/><br/> перевод с дополнениями <a href="https://thekalmanfilter.com/kalman-filter-explained-simply/#:~:text=The%20Kalman%20Filter%20uses%20the,is%20populated%20with%20differential%20equations.">этой</a> статьи</h3></div>

## Введение

Большинство учебных пособий по фильтру Калмана сложны для понимания, поскольку для понимания того, как работает фильтр Калмана, требуются продвинутые математические навыки. Если вы пытались прочитать статью <a href="http://www.cs.unc.edu/~welch/kalman/kalmanPaper.html"> «1960 Kalman Filter paper» </a> Рудольфа Э. Кальмана 1960 года, вы знаете, насколько запутанной может быть эта концепция. Но нужно ли вам понимать, как работает фильтр Калмана, чтобы его использовать?

Нет. Если вы не хотите проектировать и реализовывать фильтр Калмана, вам не нужно знать вывод всех формул, вам просто нужно понять как он работает и применяеть его на практике.

На самом деле каждый может понять фильтр Калмана, если его объяснить небольшими удобоваримыми частями. В этом посте просто объясняется фильтр Калмана и то, как он работает для оценки состояния системы.

## Обзор фильтра Калмана

Давайте посмотрим на фильтр Калмана как на черный ящик. Фильтр Калмана имеет входы и выходы. Входные данные зашумлены и иногда дают неточные измерения. Результаты его представляют собой менее зашумленные и иногда более точные оценки. Оценками могут быть параметры состояния системы, которые не измерялись и не наблюдались. Последнее предложение описывает суперсилу фильтра Калмана. Фильтр Калмана оценивает параметры системы, которые не наблюдаются и не измеряются.

Короче говоря, вы можете думать о фильтре Калмана как об алгоритме, который может оценивать наблюдаемые и ненаблюдаемые параметры с большой точностью в режиме реального времени. Оценки с высокой точностью используются для прогнозов и решений. По этим причинам фильтры Калмана используются в робототехнике и системах реального времени, которым необходима надежная информация.

Проще говоря, фильтр Калмана — это универсальный алгоритм, который используется для оценки параметров системы. Он может использовать неточные или зашумленные измерения для более точной оценки переменной или находить значения вообще не наблюдаемой переменной.

Реальную популярность фильтр Калмана обрел не благодаря способности сглаживать измерения, а благодаря своей способности оценивать параметры системы, которые невозможно измерить или наблюдать.

## Описание алгоритма 

хорошо описан <a href="https://habr.com/ru/articles/594249/">тут</a>

### Подключение библиотек

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

### Постановка задачи

Допустим у нас есть робот с датчиком на голове. Датчик возвращает кординаты Х и У через равные промежутки времени. Показания дата не точны и имеют дисперсию ss. Робот движется с постоянной не известной нам скоростью. На его координаты также влияют внешние факторы, его колеса  могут проскальзывать или застревать в грязи. Смоделируем это как шум с дисперсией ms. Задача найти скоростью с помощью фильтра Калмана. 

### Класс робот

In [2]:
class Robot:
    def __init__(self, ss, ms):
        self.ss = ss
        self.ms = ms
        self.vx = random.randint(1, 10)
        self.vy = random.randint(1, 10)
        self.x = random.randint(0, 10)
        self.y = random.randint(0, 10)

    def sn(self):
        return np.random.normal(0, self.ms, 1)[0]

        
    def move(self):
        self.x = self.vx + self.x + self.sn()
        self.y = self.vy + self.y + self.sn()

    

In [3]:
r = Robot(0.5,0.3); r.x; 

In [4]:
r.x

0

In [5]:
r.vx

2

In [6]:
r.move()

In [7]:
r.x

2.130969525159137